In [1]:
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
from ddgs import DDGS
from transformers import pipeline

def search_and_summarize(query, max_results=5):
    
    print("Revving search results...")
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

    ddgs = DDGS()
    print(f'Searching for: {query}...')
    results = ddgs.text(query, max_results=max_results)

    combined_text = ""
    for r in results:
        body = r.get('body', '')
        combined_text += body.strip().rstrip('.') + '. '

    print(f'\nRaw combined text length: {len(combined_text)} characters.')

    summary = summarizer(combined_text, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    
    return summary  

/home/leinnarf/Coding/Python/Notebook/Scrape Goat/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Usage example
search = "What is the meaning of life"
result_summary = search_and_summarize(search, max_results=5)

clean_text = result_summary.replace(" .", ".")
print(f'\nFinal Summary:\n{clean_text}')

Revving search results...


Device set to use cpu


Searching for: What is the meaning of life...

Raw combined text length: 2517 characters.

Final Summary:
 The meaning of life is the concept of an individual's life, human life, or existence in general having an inherent significance or a philosophical point. There is no consensus on the specifics of such a concept, or whether the concept itself even exists in any objective sense. Different people and cultures believe different things for the answer to this question.
